In [1]:
import numpy as np
import pandas as pd
import os
%reload_ext autoreload
%autoreload 2

# <span style='color:lightgreen'>Logs

In [2]:
lt_logs_df = pd.read_csv('./logs/long_term_forecast.csv', index_col = 0)
lt_logs_df[['model', 'data', 'seq_len', 'pred_len', 'lstm_layers', 'e_layers','d_model', 'd_layers',
       'dropout', 'learning_rate', 'nb_params', 'rmse', 'mape']]

,model,data,seq_len,pred_len,lstm_layers,e_layers,d_model,d_layers,dropout,learning_rate,nb_params,rmse,mape
0,PatchTST,ETTm2,96,96,2,3,512,1,0.1,0.0001,12617313,0.431311,1.169425
1,PatchTST,ETTm2,96,192,2,3,512,1,0.1,0.0001,13207233,0.496098,1.266275
2,PatchTST,ETTm2,96,336,2,1,512,1,0.1,0.0001,7787345,0.558475,1.400342
3,PatchTST,ETTm2,96,720,2,3,512,1,0.1,0.0001,16451793,0.645025,1.658798
4,LSTransformer,ETTm2,96,96,1,1,128,1,0.5,0.0001,17449120,0.647448,4.950500
5,LSTransformer,ETTm2,96,192,1,1,128,1,0.5,0.0001,33964864,0.719860,4.013210
6,LSTransformer,ETTm2,96,336,1,1,128,1,0.5,0.0001,58738480,1.096800,8.241722
7,LSTransformer,ETTm2,96,720,1,1,128,1,0.5,0.0001,124801456,2.427990,16.569382
8,TimeMixer,ETTm2,96,96,2,2,32,1,0.1,0.0100,237769,0.420244,1.126628
9,TimeMixer,ETTm2,96,192,2,2,32,1,0.1,0.0100,255433,0.490960,1.282125


In [3]:
st_logs_df = pd.read_csv('./logs/short_term_forecast.csv', index_col = 0)
st_logs_df[['model', 'seasonal_patterns','data', 'seq_len', 'pred_len', 'c_out', 'd_model', 'n_heads', 'e_layers', 'd_layers', 'd_ff',
       'moving_avg', 'dropout', 'learning_rate', 'avg_smape', 'avg_mase', 'avg_owa', 'avg_mape']]

,model,seasonal_patterns,data,seq_len,pred_len,c_out,d_model,n_heads,e_layers,d_layers,d_ff,moving_avg,dropout,learning_rate,avg_smape,avg_mase,avg_owa,avg_mape
0,LSTransformer,Monthly,m4,36,18,1,128,16,1,1,2048,25,0.25,0.001,85.159,18.813,8.059,66.652
1,LSTransformer,Yearly,m4,12,6,1,128,16,1,1,2048,25,0.25,0.001,85.159,18.813,8.059,66.652
2,LSTransformer,Quarterly,m4,16,8,1,128,16,1,1,2048,25,0.25,0.001,85.159,18.813,8.059,66.652
3,LSTransformer,Weekly,m4,26,13,1,128,16,1,1,2048,25,0.25,0.001,85.159,18.813,8.059,66.652
4,LSTransformer,Daily,m4,28,14,1,128,16,1,1,2048,25,0.25,0.001,85.159,18.813,8.059,66.652
5,LSTransformer,Hourly,m4,96,48,1,128,16,1,1,2048,25,0.25,0.001,85.159,18.813,8.059,66.652
6,TimesNet,Hourly,m4,96,48,1,32,8,2,1,32,25,0.10,0.001,11.920,1.598,0.857,14.017
7,Nonstationary_Transformer,Hourly,m4,96,48,1,512,8,2,1,2048,25,0.10,0.001,NaN,NaN,NaN,NaN


# <span style = 'color:lightblue'> Model

In [4]:
from types import SimpleNamespace
configs = SimpleNamespace(**lt_logs_df.loc[7].to_dict())
configs.model

'LSTransformer'

In [12]:
from models.LSTransformer import Model
from utils.model_wrapper import ModelWrapper
model = ModelWrapper(Model(configs=configs), configs)
model.summary()

Model quick summary
Layer name                    |          Nb units|      Nb tr params|  Nb non tr params
LSTM_0                        |               256|           271 360|                 0
Linear_0                      |               256|             2 048|                 0
LayerNorm_0                   |                 -|               512|                 0
Linear_1                      |               128|            32 896|                 0
TransformerEncoderLayer_0     |               128|           593 024|                 0
Linear_2                      |               256|            33 024|                 0
LayerNorm_1                   |                 -|               512|                 0
Flatten_0                     |                 -|                 0|                 0
Linear_3                      |             5 040|       123 868 080|                 0
Skip_connections : True
Transformer_nb_heads : 8
Transformer_FF_units : 2048
Dropout_rate : p = 0.5
